# **Calendari (calendari.csv)**

El següent codi crea un calendari que indica, entre d'altres, els dies laborables, caps de setmana i els festius en format boolean.

* **Font**: Elaboració pròpia

In [2]:
import datetime
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ROOT = "/content/drive/Shareddrives/GESTIÓ DE PROJECTES/X. Entregable/"
FESTIUS_FILE = ROOT + "cleaning/raw_data/festes_catalunya.csv" # csv de elaboració pròpia

In [8]:
# Anys inclosos al calendari
anys = ["2018", "2019", "2020", "2021", "2022"]

# Crear una lista de datos
data = []

# Iterar a través de los años
for any_str in anys:
    any = int(any_str)
    fecha = datetime.date(any, 1, 1)
    fecha_fin = datetime.date(any, 12, 31)

    while fecha <= fecha_fin:
        numero_dia = fecha.day
        mes = fecha.month
        nombre_dia = fecha.strftime("%A")
        es_dia_entre_semana = 1 if fecha.weekday() < 5 else 0  # 1 para días de semana, 0 para fin de semana

        # Agregar una nueva columna "Fecha_completa" con el formato "Dia/Mes/Año"
        fecha_completa = fecha.strftime("%d/%m/%Y")

        data.append([numero_dia, mes, any, nombre_dia, es_dia_entre_semana, fecha_completa])
        fecha += datetime.timedelta(days=1)

# Crear el DataFrame a partir de la lista de datos
df_fechas = pd.DataFrame(data, columns=["Numero_Dia", "Mes", "Año", "Nombre_Dia", "Es_Dia_Entre_Semana", "Fecha_Completa"])

# Leer el archivo CSV con las fechas festivas
festivos_df = pd.read_csv(FESTIUS_FILE)

festivos_df["Data"] = pd.to_datetime(festivos_df["Data"])
df_fechas["Fecha_Completa"] = pd.to_datetime(df_fechas["Fecha_Completa"])

# Crear una función que verifica si una fecha está en "festivos_df" y devuelve 1 o 0
def es_festivo(fecha):
    return 1 if fecha in festivos_df["Data"].values else 0

# Agregar la columna "Es_Festivo" en el DataFrame "df_fechas"
df_fechas["Es_Festivo"] = df_fechas["Fecha_Completa"].apply(lambda fecha: es_festivo(fecha))

def es_puente(df):
  df['Es_Puente'] = 0
  for index, row in df.iterrows():
    if row["Es_Festivo"] == 1 and row['Es_Dia_Entre_Semana'] == 1:
      if row['Nombre_Dia'] == 'Tuesday' and index > 0:
        df.at[index - 1, "Es_Puente"] = 1
      elif row['Nombre_Dia'] == 'Thursday' and index > 0:
        df.at[index + 1, "Es_Puente"] = 1
    return df

df_fechas = es_puente(df_fechas)

df_fechas["Es_Festivo"] = df_fechas["Es_Festivo"] + df_fechas["Es_Puente"]

df_fechas = df_fechas.drop(columns=["Es_Puente"])

<ipython-input-8-7908b753de01>:31: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  festivos_df["Data"] = pd.to_datetime(festivos_df["Data"])
<ipython-input-8-7908b753de01>:32: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_fechas["Fecha_Completa"] = pd.to_datetime(df_fechas["Fecha_Completa"])


In [9]:
df_fechas.to_csv(ROOT+"cleaning/clean_data/calendari.csv", index=False)